<a href="https://colab.research.google.com/github/AngeMervaneJdev/ML_AI_1/blob/main/Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,VotingRegressor
from sklearn.metrics import accuracy_score

In [3]:
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
X=np.array(data[['GrLivArea',"YearBuilt"]])
y=np.array(data.SalePrice)
y

array([208500, 181500, 223500, ..., 266500, 142125, 147500])

In [4]:
X

array([[1710, 2003],
       [1262, 1976],
       [1786, 2001],
       ...,
       [2340, 1941],
       [1078, 1950],
       [1256, 1965]])

#Probleme 1

In [5]:
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42)

In [6]:
clf1 = LinearRegression()
clf2 = DecisionTreeRegressor()
clf3 = SVR()
models=[clf1,clf2, clf3]

##1

In [8]:
#first 
predictions=[]
for ml in models:
  predictions.append(ml.fit(X_train,y_train).predict(X_test))
predictions= np.array(predictions)
MSE = mean_squared_error(y_test,np.mean(predictions,axis=0))
print('MSE : '+str(MSE))

eclf1 = VotingRegressor(estimators=[ ('lr', clf1), ('dt', clf2), ('svr', clf3)])
eclf1 = eclf1.fit(X_train, y_train)
y_pred=eclf1.predict(X_test)

MSE : 2456835551.4678335


##2

In [9]:
#second
models[2]=RandomForestRegressor();
predictions=[]
for ml in models:
  predictions.append(ml.fit(X_train,y_train).predict(X_test))
predictions= np.array(predictions)
MSE = mean_squared_error(y_test,np.mean(predictions,axis=0))
print('MSE : '+str(MSE))

eclf1 = VotingRegressor(estimators=[ ('lr', clf1), ('dt', clf2), ('rf', clf3)])
eclf1 = eclf1.fit(X_train, y_train)
y_pred=eclf1.predict(X_test)

MSE : 1758498973.0046828


comparaison with single SVR()

In [10]:
y_pred=SVR().fit(X_train,y_train).predict(X_test)

print('MSE : '+str(mean_squared_error(y_test, y_pred)))


MSE : 7171025750.700425


##3

In [20]:
#third  changing random state

models[1]=DecisionTreeRegressor(random_state=4)
models[2]=Ridge(alpha=.5)
predictions=[]
for ml in models:
  predictions.append(ml.fit(X_train,y_train).predict(X_test))
predictions= np.array(predictions)
MSE = mean_squared_error(y_test,np.mean(predictions,axis=0))
print('MSE : '+str(MSE))


eclf1 = VotingRegressor(estimators=[ ('lr', clf1), ('dt', clf2), ('svr', clf3)])
eclf1 = eclf1.fit(X_train, y_train)
y_pred=eclf1.predict(X_test)

MSE : 1970400330.232842


comparaion with LinéareRegression

In [21]:
y_pred=LinearRegression().fit(X_train,y_train).predict(X_test)

print('MSE : '+str(mean_squared_error(y_test, y_pred)))

MSE : 2314465092.732015


# Probleme 2

In [18]:
base_algorithms = [LinearRegression(), RandomForestRegressor()]

def bagging(X,y,models):
  stacking_train_dataset = []
  for model in models:
     X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,shuffle=True)    
     stacking_train_dataset.append(model.fit(X_train,y_train).predict(X_test))
  return stacking_train_dataset;

predictions=bagging(X,y,base_algorithms)

eclf1 = VotingRegressor(estimators=[ ('lr', LinearRegression()), ('rf', RandomForestRegressor())])
eclf1 = eclf1.fit(X_train, y_train)
y_pred=eclf1.predict(X_test)
print('MSE : '+str(mean_squared_error(y_test, y_pred)))

MSE : 1756349446.4038973


#Probleme 3

In [24]:
x_train, x_test, y_train, y_test=train_test_split(X,y,random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3,random_state=23)

In [30]:
def train_level_1( final_learner, train_meta_model, test_meta_model):
    # Train is carried out with final learner or meta model
    final_learner.fit(train_meta_model, y_val)

    # Getting train and test accuracies from meta_model
    print(f"Train accuracy: {final_learner.score(train_meta_model, y_val)}")
    print(f"Test accuracy: {final_learner.score(test_meta_model, y_test)}")
    return final_learner.predict(test_meta_model);


In [28]:
def train_level_0( clf):
        # Train with base x_train
        clf.fit(x_train, y_train)

        # Generate predictions for the holdout set (validation)
        # These predictions will build the input for the meta model
        val_predictions = clf.predict(x_val)

        # Generate predictions for original test set
        # These predictions will be used to test the meta model
        test_predictions = clf.predict(x_test)

        return val_predictions, test_predictions

In [44]:
weak_learners = [('dtr', DecisionTreeRegressor()),('svr', SVR()),('rf', RandomForestRegressor()),('rdg', Ridge())]

# Final learner or meta model
final_learner = LinearRegression()

train_meta_model = None
test_meta_model = None

# Start stacking
for clf_id, clf in weak_learners:

    # Predictions for each classifier based on k-fold
    val_predictions, test_predictions = train_level_0(clf)

    # Stack predictions which will form 
    # the input data for the data model
    if isinstance(train_meta_model, np.ndarray):
        train_meta_model = np.vstack((train_meta_model, val_predictions))
    else:
        train_meta_model = val_predictions

    # Stack predictions from test set
    # which will form test data for meta model
    if isinstance(test_meta_model, np.ndarray):
        test_meta_model = np.vstack((test_meta_model, test_predictions))
    else:
        test_meta_model = test_predictions

# Transpose train_meta_model
train_meta_model = train_meta_model.T

# Transpose test_meta_model
test_meta_model = test_meta_model.T

# Training level 1
train_level_1(final_learner, train_meta_model, test_meta_model)

#### my sigle model Ridge()

mdl=Ridge().fit(x_train,y_train)
pred=mdl.predict(x_val)
mdl.score(x_val,y_val)

Train accuracy: 0.643735885697504
Test accuracy: 0.7258201444502002


0.568983383927375